In [6]:
import folium #Umożliwia tworzenie interaktywnych map w języku Python
from IPython.display import display # Umożliwia wyświetlanie interaktywnych wyników w środowisku IPython
from geopy.geocoders import Nominatim #służy do geokodowania, czyli przekształcania adresów lub miejsc na współrzędne geograficzne.
import requests
import polyline # używana do reprezentacji tras na mapie.
import pandas as pd

In [11]:
data = pd.read_csv('./sonar_training.csv', index_col=0)
data['ulica'] = data['ulica'].astype(str) + ', Wrocław'
streets = list(data.ulica.unique())
len(streets)

700

In [8]:
mapbox_api_key = 'pk.eyJ1Ijoid3ljZW55IiwiYSI6ImNscnRraDBsajA3NXQyam5hdjloZTBoZ3EifQ.0gU_ozTPH70_eXfyfCAqkA'

geolocator = Nominatim(user_agent="my_geocoder") #definiujemy usługę geokodowania
addresses = streets#, 'Zamkowa 1, Wrocław']

loc = geolocator.geocode('Wrocław')
loc = [loc.latitude,loc.longitude]

map_wroclaw = folium.Map(loc, zoom_start=13, control_scale=True)

# Dodawanie warstwy mapbox do mapy
folium.TileLayer(
    tiles=f'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}',
    attr='Mapbox',
    name='Mapbox Streets',
    overlay=True
).add_to(map_wroclaw)
folium.LayerControl().add_to(map_wroclaw)
display(map_wroclaw)

In [9]:
for address in addresses:
    location = geolocator.geocode(address)
    folium.Marker(
        location=[location.latitude, location.longitude],
        popup=address,
        icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(map_wroclaw)
display(map_wroclaw)

In [10]:
stop

NameError: name 'stop' is not defined

In [ ]:
#https://docs.mapbox.com/api/navigation/directions/
start_location = geolocator.geocode(addresses[0])
end_location = geolocator.geocode(addresses[1])
start_coordinates = f'{start_location.longitude},{start_location.latitude}'
end_coordinates = f'{end_location.longitude},{end_location.latitude}'
#driving/walking/cycling
url = f'https://api.mapbox.com/directions/v5/mapbox/driving/{start_coordinates};{end_coordinates}?access_token={mapbox_api_key}'
response = requests.get(url)
data = response.json()
print(data.keys())
print(data.get('routes'))

dict_keys(['routes', 'waypoints', 'code', 'uuid'])
[{'weight_name': 'auto', 'weight': 5417.075, 'duration': 693.536, 'distance': 3190.557, 'legs': [{'via_waypoints': [], 'admins': [{'iso_3166_1_alpha3': 'POL', 'iso_3166_1': 'PL'}], 'weight': 5417.075, 'duration': 693.536, 'steps': [], 'distance': 3190.557, 'summary': 'Plac Grunwaldzki, Oławska'}], 'geometry': 'wn}vHsyagBSwBsEwCa@y@wCcALk@a@y@xCwMjAcBjIfb@rEn\\zGt^iAxMw@dUzBbNDvIe@|GkEnZEpBjCuN|@j@'}]


In [ ]:
data

{'routes': [{'weight_name': 'auto',
   'weight': 5417.075,
   'duration': 693.536,
   'distance': 3190.557,
   'legs': [{'via_waypoints': [],
     'admins': [{'iso_3166_1_alpha3': 'POL', 'iso_3166_1': 'PL'}],
     'weight': 5417.075,
     'duration': 693.536,
     'steps': [],
     'distance': 3190.557,
     'summary': 'Plac Grunwaldzki, Oławska'}],
   'geometry': 'wn}vHsyagBSwBsEwCa@y@wCcALk@a@y@xCwMjAcBjIfb@rEn\\zGt^iAxMw@dUzBbNDvIe@|GkEnZEpBjCuN|@j@'}],
 'waypoints': [{'distance': 13.188,
   'name': '',
   'location': [17.053861, 51.110357]},
  {'distance': 15.217, 'name': 'Zamkowa', 'location': [17.029619, 51.107492]}],
 'code': 'Ok',
 'uuid': 'LSvibEG01zrNGbunaVW94WWbI19Uc_RwMG66k0dhVH5sN9z82hNHVw=='}

In [ ]:
coordinates = polyline.decode(data['routes'][0]['geometry'])
folium.PolyLine(locations=coordinates, color='blue').add_to(map_wroclaw)
folium.LayerControl().add_to(map_wroclaw)
display(map_wroclaw)

In [ ]:
travel_time = data['routes'][0]['duration'] / 60  # czas w minutach
travel_time

11.558933333333332

In [ ]:
#Załadowanie mapy od nowa
map_wroclaw = folium.Map(loc, zoom_start=13, control_scale=True)
folium.TileLayer(
    tiles=f'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}',
    attr='Mapbox',
    name='Mapbox Streets',
    overlay=True
    ).add_to(map_wroclaw)
geolocator = Nominatim(user_agent="my_geocoder") 
#Tworzenie nowych punktów na mapie
addresses = ["Instytut Matematyczny, Wrocław", "Swobodna 1, Wrocław", "Galeria Dominikańska, Wrocław"]
for address in addresses:
    location = geolocator.geocode(address)
    folium.Marker(
        location=[location.latitude, location.longitude],
        popup=address,
        icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(map_wroclaw)
folium.LayerControl().add_to(map_wroclaw)
display(map_wroclaw)

In [ ]:
coordinates = [(geolocator.geocode(address).longitude, geolocator.geocode(address).latitude) for address in addresses]

In [ ]:
#Szukanie optymalnej trasy
profile = 'driving'
optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]}?access_token={mapbox_api_key}'
response = requests.get(optimized_trips_url)
data_opt = response.json()
print(f"Odległość trasy: ok. {round(data_opt['trips'][0]['distance']/1000,2)} km")
print(f"Czas podróży: ok. {round(data_opt['trips'][0]['duration']/60)} min")

Odległość trasy: ok. 8.1 km
Czas podróży: ok. 36 min


In [ ]:
#szukanie trasy bez warunku optymalności
trips_url = f'https://api.mapbox.com/directions/v5/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]}?access_token={mapbox_api_key}'
response = requests.get(trips_url)
data = response.json()
for route in data['routes']:
    print(f"Odległość trasy: {round(route['distance']/1000)} km")
    print(f"Czas podróży: {round(route['duration']/60)} min")

Odległość trasy: 6 km
Czas podróży: 23 min


In [ ]:
coordinates = polyline.decode(data_opt['trips'][0]['geometry'])
folium.PolyLine(locations=coordinates, color='red').add_to(map_wroclaw)
coordinates = polyline.decode(data['routes'][0]['geometry'])
folium.PolyLine(locations=coordinates, color='blue').add_to(map_wroclaw)
display(map_wroclaw)

In [ ]:
profile = 'driving'
optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]}?source=first&destination=last&roundtrip=false&access_token={mapbox_api_key}'
response = requests.get(optimized_trips_url)
data_opt = response.json()
print(f"Odległość trasy: ok. {round(data_opt['trips'][0]['distance']/1000,2)} km")
print(f"Czas podróży: ok. {round(data_opt['trips'][0]['duration']/60)} min")
coordinates = polyline.decode(data_opt['trips'][0]['geometry'])
folium.PolyLine(locations=coordinates, color='blue').add_to(map_wroclaw)
display(map_wroclaw)

Odległość trasy: ok. 0.0 km
Czas podróży: ok. 0 min
